# Messing around with location history

In [1]:
#!pip install HASS-data-detective # Install detective

In [2]:
!pip show HASS-data-detective

Name: HASS-data-detective
Version: 2.4
Summary: Tools for studying Home Assistant data.
Home-page: https://github.com/robmarkcole/HASS-data-detective
Author: Robin Cole
Author-email: robmarkcole@gmail.com
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: pandas, pytz, ruamel.yaml, SQLAlchemy
Required-by: 


In [3]:
import detective.core as detective
import detective.functions as functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

db = detective.db_from_hass_config()

YAML tag !include_dir_merge_list is not supported
YAML tag !include_dir_merge_named is not supported
Successfully connected to database sqlite:////config/home-assistant_v2.db
There are 348 entities with data


First we extract all state changes for Gaby and Charles

In [12]:
people = ("person.charles",)

In [16]:
results = db.fetch_all_data_of(("sensor.bedroom_blinds_power",))
# results = db.fetch_all_sensor_data()
# results.last_changed = pd.to_datetime(results.last_changed)
# results.set_index('last_changed', inplace=True)
# results.index.rename("time", inplace=True)
# results.sort_index(inplace=True)
results

The returned Pandas dataframe has 397 rows of data.


,domain,entity_id,state,last_changed,attributes
0,None,sensor.bedroom_blinds_power,38,2022-09-03 08:53:01.739082,None
1,None,sensor.bedroom_blinds_power,38,2022-09-01 22:37:28.420258,None
2,None,sensor.bedroom_blinds_power,38,2022-09-01 22:37:28.420258,None
3,None,sensor.bedroom_blinds_power,38,2022-09-01 22:37:28.420258,None
4,None,sensor.bedroom_blinds_power,39,2022-09-01 21:35:32.170971,None
...,...,...,...,...,...
392,None,sensor.bedroom_blinds_power,50,2022-05-07 11:13:47.121119,None
393,None,sensor.bedroom_blinds_power,50,2022-04-30 10:22:48.092602,None
394,None,sensor.bedroom_blinds_power,50,2022-04-30 10:22:48.092602,None
395,None,sensor.bedroom_blinds_power,50,2022-04-30 10:22:48.092602,None


In [18]:
query = str(
    """
    SELECT DISTINCT(entity_id) FROM states
    """
)
response = self.perform_query(query)

NameError: name 'self' is not defined

In [10]:
results['attributes']

time
2022-05-05 20:13:55.110599    None
2022-05-05 20:13:55.110599    None
2022-05-05 20:13:55.110599    None
2022-05-05 20:13:55.110599    None
2022-05-05 20:13:55.110599    None
                              ... 
2022-09-02 23:03:40.012767    None
2022-09-03 08:52:23.430499    None
2022-09-03 08:52:23.430499    None
2022-09-03 08:52:23.431763    None
2022-09-03 08:52:23.431763    None
Name: attributes, Length: 26786, dtype: object

Person states have a lot of duplicates due to their attributes. Drop these, and use the times as the index:

In [6]:
results['time'] = results.index
results = results.drop_duplicates(subset=('domain', 'entity_id', 'state', 'time')).drop(['time', 'attributes'], axis=1)
results

,domain,entity_id,state
time,,,
2022-05-05 20:13:55.110599,None,person.gaby,home
2022-05-05 20:14:25.356517,None,person.charles,home
2022-05-06 06:55:21.896916,None,person.gaby,not_home
2022-05-06 07:10:36.457771,None,person.gaby,Central london
2022-05-06 07:29:27.015025,None,person.charles,not_home
...,...,...,...
2022-09-02 22:59:00.936924,None,person.gaby,not_home
2022-09-02 23:02:04.993106,None,person.gaby,home
2022-09-02 23:03:40.012767,None,person.charles,home


For each day, if the person went to work, store their arrival & departure times

In [7]:
dates = pd.date_range(min(results.index), max(results.index), normalize=True)

arrival_times = {person: pd.Series(dtype="timedelta64[ns]") for person in people}
leaving_times = {person: pd.Series(dtype="timedelta64[ns]") for person in people}

for date in dates:
    state_changes = results[date : date + np.timedelta64(1, 'D')]
    
    for person, workplace in zip(people, workplaces):
        this_person_changes = state_changes[state_changes.entity_id == person]
        if any(this_person_changes.state == workplace):
            workplace_states_ind = np.flatnonzero(this_person_changes.state == workplace)
            
            arrival_times[person][date] = this_person_changes.index[min(workplace_states_ind)] - date
            leaving_times[person][date] = this_person_changes.index[max(workplace_states_ind) + 1] - date
            
arrival_times = pd.DataFrame(arrival_times)
leaving_times = pd.DataFrame(leaving_times)

NameError: name 'workplaces' is not defined

We're only interested in the morning and evening, so filter out WFH events or drunken evenings

In [ ]:
arrival_times[arrival_times < np.timedelta64(5,'h')] = None
arrival_times[arrival_times > np.timedelta64(12,'h')] = None
leaving_times[leaving_times < np.timedelta64(12,'h')] = None

working_hours = (leaving_times - arrival_times)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

(arrival_times / np.timedelta64(1, 'h')).plot.hist(title="Arrival times at work", grid=True, alpha = 0.7)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

(leaving_times / np.timedelta64(1, 'h')).plot.hist(title="Departure times from work", grid=True, alpha = 0.7)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

(working_hours / np.timedelta64(1, 'h')).plot.hist(title="Total hours at work", grid=True, alpha = 0.7)

In [ ]:
arrival_times.mean()

In [ ]:
leaving_times.mean()

In [ ]:
working_hours.mean()

In [ ]:
arrival_times.median()

In [ ]:
leaving_times.median()

In [ ]:
working_hours.median()